## Imports

In [1]:
# for loading the data
import cookielib
import urllib
import urllib2
import getpass
import sys
import json
# standard imports
from collections import defaultdict, OrderedDict
from copy import deepcopy
import math
# third party packages
import numpy as np

## Load Data

In [2]:
HOST = "https://working-dog-data-dash.appspot.com"

def create_opener():
    """creates a urllib2.OpenerDirector with a CookieJar"""
    cj = cookielib.CookieJar()
    opener = urllib2.build_opener(urllib2.HTTPCookieProcessor(cj))
    return opener

def login(opener):
    """prompts the cli user for login credentials to P.A.W.S.
     then sends a login request to the real server using opener.

    Arguments:
        opener - a urllib2.OpenerDirector, this should have a cookier jar.
    """
    login_url = HOST + "/login"
    print("Please enter credentials for P.A.W.S.")
    user = getpass.getpass("Userame: ")
    password = getpass.getpass("Password: ")
    form_data = {"username": user, "password": password}
    params = urllib.urlencode(form_data)
    response = opener.open(login_url, data=params)
    code = response.getcode()
    if code != 200:
        raise Exception("Failed to login!")

# login to backend service
opener = create_opener()
login(opener)

# fetch data from server
url = HOST + "/api/cached/data/filtered/blob"
response = opener.open(url)

# read and check result
raw = response.read()
if raw[0] != '{':
    raise Exception("Failed to load!")

# parse the JSON
# NOTE: we have no floats
# The data format is at:
# https://github.com/ChelsiAlise/workingdogdatadash/blob/master/data.go
blob = json.loads(raw, parse_int=int, parse_float=int)

# Debug loaded data
num_dogs = len(blob["dogs"])
print ""
print "Loaded data with num_dogs = %d" % (num_dogs)

Please enter credentials for P.A.W.S.
Userame: ········
Password: ········

Loaded data with num_dogs = 124


## Clean Data

In [3]:
# define small common utilities
def print_bar():
    print "-"*70

In [4]:
# make dict of data by dog id
dogs = {}
for dog in blob["dogs"]:
    id = dog["id"]
    dogs[id] = dog
    dogs[id]["days"] = []

# copy day info to dogs
for day in blob["days"]:
    date = day["date"]
    for day_data in day["dogs"]:
        id = day_data["id"]
        dogs[id]["days"].append(day_data)
        dogs[id]["days"][-1]["date"] = date

print "Number of days (filtered to 70% of a day) least to most:"
print_bar()
print sorted([len(dogs[id]["days"]) for id in dogs])

Number of days (filtered to 70% of a day) least to most:
----------------------------------------------------------------------
[27, 39, 39, 44, 45, 48, 50, 57, 62, 69, 71, 77, 83, 84, 85, 86, 87, 91, 100, 127, 140, 143, 145, 161, 175, 206, 221, 227, 230, 234, 252, 256, 260, 265, 283, 300, 301, 302, 319, 328, 337, 338, 342, 346, 361, 362, 364, 366, 366, 373, 374, 377, 380, 384, 399, 401, 404, 415, 422, 447, 448, 451, 456, 465, 470, 478, 479, 480, 484, 484, 485, 488, 493, 495, 498, 500, 502, 503, 503, 504, 509, 510, 513, 517, 520, 528, 529, 533, 533, 534, 534, 536, 539, 539, 541, 549, 558, 561, 564, 567, 576, 596, 600, 603, 606, 608, 617, 619, 619, 619, 629, 634, 636, 643, 644, 647, 649, 649, 652, 671, 673, 674, 678, 681]


In [5]:
DAY_MINS = 1440

# get and sort all totals for analysis
totals = sorted([dog["total"] for dog in blob["dogs"]])
print "Minute Totals:"
print_bar()
total_mean = np.mean(totals)
total_std = np.std(totals)
total_ten_pct = totals[int(len(totals)*.1)]
# print helper
def print_value_days_weeks(name, value):
    print "%10s: %10f (mins), %10f (days), %10f (weeks)" %\
        (name, value, value/DAY_MINS, value/DAY_MINS/7)
print_value_days_weeks("Mean", total_mean)
print_value_days_weeks("Std-Dev", total_std)
print_value_days_weeks("Bottom 10%", total_ten_pct)
print_value_days_weeks("Mean - Std", total_mean - total_std)
print_bar()

Minute Totals:
----------------------------------------------------------------------
      Mean: 560912.879032 (mins), 389.522833 (days),  55.646119 (weeks)
   Std-Dev: 274515.776789 (mins), 190.635956 (days),  27.233708 (weeks)
Bottom 10%: 116940.000000 (mins),  81.000000 (days),  11.000000 (weeks)
Mean - Std: 286397.102243 (mins), 198.886877 (days),  28.412411 (weeks)
----------------------------------------------------------------------


In [6]:
# filter to have at least 1/2 year of valid days
total_threshold = 26 * 7

filtered_dogs = {}
removed_dogs_by_outcome = defaultdict(list)
for k, v in dogs.iteritems():
    if len(v["days"]) >= total_threshold:
        filtered_dogs[k] = deepcopy(v)
    else:
        status = v["dog_status"]
        if not status: status = "Unkown Status"
        removed_dogs_by_outcome[status].append(v)

num_below_threshold = len(dogs) - len(filtered_dogs)

print "Number of dogs below threshold: %d" % (num_below_threshold)
print "      Number of remaining dogs: %d" % (len(filtered_dogs))

print_bar()

for k, v in removed_dogs_by_outcome.iteritems():
    print "Outcome: %18s   => %3d dogs removed." % ("'"+k+"'", len(v))

Number of dogs below threshold: 25
      Number of remaining dogs: 99
----------------------------------------------------------------------
Outcome:   'Active Breeder'   =>   4 dogs removed.
Outcome:    'Unkown Status'   =>   7 dogs removed.
Outcome:  'Active Grad Dog'   =>   8 dogs removed.
Outcome:     'Released Dog'   =>   6 dogs removed.


## Analyze Data

In [8]:
# first put the dog data into buckets by outcome
dogs_by_outcome = defaultdict(list)
for dog in filtered_dogs.itervalues():
    status = dog["dog_status"]
    if status == "":
        status = "Unknown Status"
    dogs_by_outcome[status].append(dog)

# create alphabetically sorted set of possible outcomes
outcomes = sorted(dogs_by_outcome.keys())

print "Outcomes:"
print_bar()

# analyze dogs by outcome
results_by_outcome = {}
for outcome in outcomes:
    results = OrderedDict()
    results["outcome"] = outcome,
    results["dogs"] = []
    # convert to percentages instead of totals
    for dog in dogs_by_outcome[outcome]:
        result = deepcopy(dog)
        total = dog["total"]
        result["active_total"] = dog["active"]
        result["active"] = float(result["active_total"]) / total * 100
        result["rest_total"] = dog["rest"]
        result["rest"] = float(result["rest"]) / total * 100
        result["awake_total"] = dog["awake"]
        result["awake"] = float(result["awake_total"]) / total * 100
        results["dogs"].append(result)
    # compute means over outcome
    active = [v["active"] for v in results["dogs"]]
    awake = [v["awake"] for v in results["dogs"]]
    rest = [v["rest"] for v in results["dogs"]]
    results["num_dogs"] = len(active)
    results["active_mean"] = np.mean(active)
    results["active_std"] = np.std(active)
    results["awake_mean"] = np.mean(awake)
    results["awake_std"] = np.std(awake)
    results["rest_mean"] = np.mean(rest)
    results["rest_std"] = np.std(rest)
    # print results for outcome
    print "     Outcome:  " + outcome
    for key in results.keys():
        if key not in ["dogs", "outcome"]:
            print "%12s: %10f" % (key, results[key])
    print_bar()
    results_by_outcome[outcome] = results


Outcomes:
----------------------------------------------------------------------
     Outcome:  Active Breeder
    num_dogs:   6.000000
 active_mean:   6.526844
  active_std:   0.952408
  awake_mean:  32.026897
   awake_std:   1.896043
   rest_mean:  61.446259
    rest_std:   1.611735
----------------------------------------------------------------------
     Outcome:  Active Grad Dog
    num_dogs:  44.000000
 active_mean:   6.780170
  active_std:   2.070600
  awake_mean:  31.895558
   awake_std:   2.712022
   rest_mean:  61.324272
    rest_std:   3.603033
----------------------------------------------------------------------
     Outcome:  Advanced Training
    num_dogs:   5.000000
 active_mean:   6.828395
  active_std:   0.732784
  awake_mean:  31.230064
   awake_std:   2.429241
   rest_mean:  61.941541
    rest_std:   2.831610
----------------------------------------------------------------------
     Outcome:  Released Dog
    num_dogs:  38.000000
 active_mean:   6.627658
  active_